# LAB1: Object detection
### Please write a Python code which realize following functionality:
1. The software should read a movie file of given name (should read the name as a runtime parameter),
2. It should mark on the screen red object and follow it during all time (circle of proper size)
3. Small artifacts (extra dots and holes) should be removed from the screen.
4. The software should open two windows, one with movie without transformation and the second with movie transformed and thresholded
5. On the screen with no transformed movie red object should be marked and left/right
### deviation from the center of the pictures should be inserted.
### The task should be sent by Teams system according to the dates specified in the assignment.
#### Good luck
Mateusz Andrzejewski

---

### Created by: `Jakub Góralczyk`

---

In [1]:
from collections import deque
import cv2
import numpy as np

In [2]:
def object_detection(video_string: str = 'videos\\F1_r.MOV'):
    video = cv2.VideoCapture(video_string)
    if video is None:
        print(f'Unable to open video at {video_string}')
        return

    video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    writer = cv2.VideoWriter('videos\\original.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(video_width,video_height))
    writer2 = cv2.VideoWriter('videos\\transformed.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(video_width,video_height))
    frame_grabbed, frame = video.read()

    frame_size = frame.shape[:1]
    frame_width = video.get(3)


    lower = (155,25,0) #lower bound for red color
    upper = (179,255,255) #upper bound for red color

    # value -> ksize
    value = 14
    while frame_grabbed:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # convert to HSV

        mask = cv2.inRange(hsv,lower,upper) # create mask from lower to upper bounds

        mask = cv2.erode(mask,cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(value,value))) # erode mask based on structuring element size 14 x 14 pixels ellipse
        mask = cv2.dilate(mask,cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(value,value)))# dilate mask based on structuring element size 14 x 14 pixels ellipse

        mask = cv2.dilate(mask,cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(value,value))) # dilate mask based on structuring element size 14 x 14 pixels ellipse
        mask = cv2.erode(mask,cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(value,value))) # erode mask based on structuring element size 14 x 14 pixels ellipse

        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2] # find contours in mask

        #center is a value for the middle of the object
        center = None 


        if len(contours) > 0: 
            #getting the position and radius for the circle
            c = max(contours, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            # center stores x and y values of the center poin of the tracked object
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

            if radius > 20:
                #creates a circle around the object
                cv2.circle(frame, (int(x), int(y)), int(radius),
				(0, 255, 255), 2)


        #line follows from the middle of a window to the center of the tracked object
        cv2.line(frame, (int(center[0]), 10), (int(frame_width/2), 10), (255, 255, 255), thickness=5)
        #cv2.imshow('original',frame)
        #cv2.imshow('transformed',mask)

        writer.write(frame)
        mask  = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
        writer2.write(mask)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        frame_grabbed, frame = video.read()
    writer.release()
    writer2.release()
    video.release()
    cv2.destroyAllWindows()

In [3]:
object_detection()